In [19]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import ChromeOptions
import time
import re

In [20]:
option = ChromeOptions()
option.add_argument('user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
browser=webdriver.Chrome(executable_path="/Applications/Google Chrome.app/Contents/MacOS/chromedriver", options=option)

In [21]:
browser.get("https://www.damai.cn/?utm_source=search&utm_medium=baidupc&utm_content=pmarket_yhcpyyb&utm_campaign=alink_bdsem_shh_shouye_8000")

In [22]:
%%time
def is_element_exist(query_type, value):
    """根据查询类型判断页面中是否存在相应的元素"""
    try:
        if query_type == "css":
            if len(browser.find_elements_by_css_selector(value)) > 0:
                return True
            else:
                return False
        elif query_type == "class":
            if len(browser.find_elements_by_class_name(value)) > 0:
                return True
            else:
                return False
        elif query_type == "id":
            if len(browser.find_elements_by_id(value)) > 0:
                return True
            else:
                return False
        elif query_type == "xpath":
            if len(browser.find_elements_by_xpath(value)) > 0:
                return True
            else:
                return False
        else:
            return "query type is error, only support css, class, id, xpath!!!"
    except:
        return False

CPU times: user 6 µs, sys: 3 µs, total: 9 µs
Wall time: 12.9 µs


In [23]:
def login():
    """登录"""
    """大麦会识别出非人工操作，从输入用户名密码开始到验证码结束人工操作"""
    #点击登录按钮,跳转到登录界面
    if browser.title=="大麦网-全球演出赛事官方购票平台-100%正品、先付先抢、在线选座！":
        browser.find_element_by_css_selector(".login-user").click()
        browser.implicitly_wait(10)
        #进入登录的iframe，坑，竟然在一个大页面中有一个iframe
        browser.switch_to_frame("alibaba-login-box")
    browser.find_element_by_id("fm-login-id").clear()
    #输入用户名，需要替换成自己的用户名
    browser.find_element_by_id("fm-login-id").send_keys("")
    browser.find_element_by_id("fm-login-password").clear()
    #输入密码
    browser.find_element_by_id("fm-login-password").send_keys("")
    #判断是否存在滑动解锁
    #print(is_element_exist("class", "nc-lang-cnt"))
    # TODO 验证失败
    if is_element_exist("class", "nc-lang-cnt"):
        #滑动验证码,获取平移距离,移动解锁
        start=browser.find_element_by_id("nc_1_n1z")
        inner_size=start.size["width"]
        outer_size=browser.find_element_by_class_name("nc-lang-cnt").size["width"]
        move_size=outer_size-inner_size
        ActionChains(browser).drag_and_drop_by_offset(start, move_size, 0).perform()
        time.sleep(2)#滑完了之后稍等下，让系统判断完毕

    browser.find_element_by_css_selector(".password-login").click()

In [24]:
login_count=1
def login_continue():
    """"""
    global login_count
    try:
        if (login_count > 10):
            print("登录次数超过10次，请检查程序是否有误")
        else:
            login()
    except:
        login_count = login_count + 1
        login_continue()

In [25]:
# perl -pi -e 's/cdc_/dog_/g' /Applications/chromedriver
login_continue()

/Users/admin/Software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use driver.switch_to.frame instead
  if __name__ == '__main__':


In [26]:
def search(performer, city=None):
    """登录后首页搜索，可以选择城市"""
    if city != None:
        current_location = browser.find_element_by_class_name("city-location")
        ActionChains(browser).move_to_element(current_location).perform()
        cities = browser.find_elements_by_class_name("name-city")
        for city in cities:
            if city.text == city:
                city.click()
                browser.implicitly_wait(20)
                break 
    browser.find_element_by_class_name("input-search").clear()
    browser.find_element_by_class_name("input-search").send_keys(performer)
    browser.find_element_by_class_name("btn-search").click()
    browser.implicitly_wait(20)

In [27]:
search("杨千嬅","海口")

In [28]:
# 这段代码没啥用
def choose_city_after_search_people(city=None):
    """搜索明星后选择城市"""
    if city != None:
        cities = browser.find_elements_by_class_name("factor-content-item")
        for city in cities:
            if city.text == city:
                city.click()
                browser.implicitly_wait(20)
                break 

In [29]:
def buy_detail(city):
    """进入购买详情页"""
    browser.find_element_by_partial_link_text(city).click()
    browser.implicitly_wait(20)

In [30]:
buy_detail("海口")

In [31]:
# 切换窗口
search_result_window = browser.current_window_handle
buy_page_window = None
for window in browser.window_handles:
    if window != browser.current_window_handle:
        buy_page_window = window
        browser.switch_to_window(window)
        break

/Users/admin/Software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use driver.switch_to.window instead
  import sys


In [32]:
def buy_now(total):
    """total为购买票的数目"""
    
    # 判断限购数量
    try:
        limit = re.search("\d",browser.find_element_by_class_name("number_right_limit").text).group(0)
        if total > int(limit):
            total=limit
        browser.find_element_by_class_name("cafe-c-input-number-input").clear()
        browser.find_element_by_class_name("cafe-c-input-number-input").send_keys(total)
        browser.find_element_by_class_name("buybtn").click()
        browser.implicitly_wait(10)
    except:
        print("获取限购数量失败,直接使用输入票数【"+str(total)+"】")
        browser.find_element_by_class_name("cafe-c-input-number-input").clear()
        browser.find_element_by_class_name("cafe-c-input-number-input").send_keys(total)
        browser.find_element_by_class_name("buybtn").click()
        browser.implicitly_wait(10)
    
    
    


In [33]:
def choose_time_and_price(price, start_time_sort_num):
    """选择价位与时间"""
    """start_time_sort_num时间顺序，页面上的排序，开始第一个数是1"""
    # 选择时间
    all_items = browser.find_elements_by_class_name("select_right_list_item")
    sorts=[]
    price_levels=[]
    for item in all_items:
        if item.get_attribute("class").find("sku_item") == -1:
            sorts.append(item)
        else:
            price_levels.append(item)
    if len(sorts) > 1:
        for index,sort in enumerate(sorts,1):
            if index == start_time_sort_num:
                sort.click()
                break
                
    # 选择价位
    if len(price_levels) > price:
        print("输入价位信息错误，超过系统的价位档,将被设置成最低档价位的票")
        price = 1 
    price_levels[price-1].click()
    #for each_price in price_levels:
    #    if each_price.text.find(str(price) + "元") >  -1:
    #        each_price.click()
    #        break
            
    #返回是否可以购买
    return browser.find_element_by_class_name("buybtn").text

In [34]:
# TODO 费时间
def refresh_and_buy(price, total, start_time_sort_num):
    """判断是否开售，未开售就刷新浏览器"""
    """price：价格"""
    """total：选择抢票数目"""
    # 是否可以购票
    txt = choose_time_and_price(price, start_time_sort_num)
    while txt=="即将开抢":
        browser.refresh()
        print("即将开抢")
        txt = choose_time_and_price(price, start_time_sort_num)
    if txt == "提交缺货登记":
        print("票已被抢完")
        exit()
    buy_now(total)
    
    # 点击立即购买出错跳到页面，需要添加判断http://err.taobao.com/error2.html
    while browser.current_url == "http://err.taobao.com/error2.html":
        browser.back()
        refresh_and_buy(price, total, start_time_sort_num)

In [35]:
%%time
refresh_and_buy(398,2,1)

即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢
即将开抢


KeyboardInterrupt: 

In [17]:
def submit_order():
    """提交订单"""
    browser.find_element_by_class_name("next-checkbox-label").click()
    browser.find_elements_by_class_name("next-btn-medium")[2].click()
    
    # send email

In [18]:
submit_order()

In [27]:
%%time
is_element_exist("class","buybtn")

CPU times: user 1.59 ms, sys: 1.44 ms, total: 3.02 ms
Wall time: 17.4 ms


True

In [28]:
%%time
buy_now(1280,1,1)

CPU times: user 20.2 ms, sys: 3.44 ms, total: 23.7 ms
Wall time: 8.41 s


In [39]:
def buy_now(price, total, start_time_sort_num):
    import time
    a = time.time()
    all_items = browser.find_elements_by_class_name("select_right_list_item")
    sorts=[]
    price_levels=[]
    for item in all_items:
        if item.get_attribute("class").find("sku_item") == -1:
            sorts.append(item)
        else:
            price_levels.append(item)
    if len(sorts) > 1:
        for index,sort in enumerate(sorts,1):
            if index == start_time_sort_num:
                sort.click()
                break
    b=time.time()
    print(b-a)
    # 选择价位
    for each_price in price_levels:
        if each_price.text.find(str(price) + "元") >  -1:
            each_price.click()
            break

    c=time.time()
    print(c-b)
    # 判断限购数量
    limit = re.search("\d",browser.find_element_by_class_name("number_right_limit").text).group(0)
    if total > int(limit):
        total=limit
    if total != 1:
        browser.find_element_by_class_name("cafe-c-input-number-input").clear()
        browser.find_element_by_class_name("cafe-c-input-number-input").send_keys(total)
    browser.find_element_by_class_name("buybtn").click()

    d=time.time()
    print(d-c)
    # 点击立即购买出错跳到页面，需要添加判断http://err.taobao.com/error2.html
    while browser.current_url == "http://err.taobao.com/error2.html":
        browser.back()
        buy_now(price,total,start_time_sort_num)
    e=time.time()
    print(e-d)

In [42]:
%%time
buy_now(1280,2,1)

CPU times: user 31.2 ms, sys: 4.24 ms, total: 35.5 ms
Wall time: 2.05 s
